In [39]:
import pandas as pd 
import numpy as np
df  = pd.read_csv('/Users/rishitkapoor/Documents/Python_ML/Ch_5_Custering/imdb.csv')
df.head()

,title,year,kind,genre,rating,vote,country,language,cast,director,composer,writer,runtime
0,Dinosaur Planet,2003.0,tv mini series,"['Documentary', 'Animation', 'Family']",7.7,474.0,['United States'],['English'],"['Christian Slater', 'Scott Sampson']",NaN,NaN,"['Mike Carrol', 'Mike Carroll', 'Georgann Kane']",50.0
1,Character,2021.0,movie,"['Crime', 'Horror', 'Thriller']",8.3,46.0,['Japan'],['Japanese'],"['Masaki Suda', 'Fukase', 'Mitsuki Takahata', ...",['Akira Nagai'],['Youki Kojima'],"['Takashi Nagasaki', 'Takashi Nagasaki', 'Anna...",125.0
2,Get Up and Dance!,1994.0,video movie,['Family'],8.1,18.0,['United States'],['English'],"['Paula Abdul', 'Aurorah Allain', 'Bill Bohl',...",['Steve Purcell'],NaN,NaN,54.0
3,The Rise and Fall of El Chapo,2016.0,tv movie,['Documentary'],6.9,42.0,['United States'],NaN,NaN,NaN,NaN,NaN,85.0
4,Sick - IMDb,NaN,NaN,['Thriller'],NaN,NaN,['United States'],NaN,"['Marc Menchaca', 'Gideon Adlon', 'Dylan Spray...",['John Hyams'],NaN,"['Katelyn Crabb', 'Kevin Williamson']",NaN


In [40]:
df = df.drop(['director','kind','cast','runtime','composer','year','writer'],axis=1)
df.head()

,title,genre,rating,vote,country,language
0,Dinosaur Planet,"['Documentary', 'Animation', 'Family']",7.7,474.0,['United States'],['English']
1,Character,"['Crime', 'Horror', 'Thriller']",8.3,46.0,['Japan'],['Japanese']
2,Get Up and Dance!,['Family'],8.1,18.0,['United States'],['English']
3,The Rise and Fall of El Chapo,['Documentary'],6.9,42.0,['United States'],NaN
4,Sick - IMDb,['Thriller'],NaN,NaN,['United States'],NaN


In [41]:
from pandas import DataFrame
ad =pd.DataFrame(df[['genre']],columns =['genre'])
ad.head()

,genre
0,"['Documentary', 'Animation', 'Family']"
1,"['Crime', 'Horror', 'Thriller']"
2,['Family']
3,['Documentary']
4,['Thriller']


In [42]:
# General defination of preprocessing for test data
import unicodedata
import string
import textblob
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

class Preprocess():
    def remove_accented_chars(text):
        a = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return a

    def simple_text_preprocessor(document): 
        # lower case
        document = str(document).lower()
        
        # expand contractions
        document = contractions.fix(document)
        
        # remove unnecessary characters
        document = re.sub(r'[^a-zA-Z]',r' ', document)
        document = re.sub(r'nbsp', r'', document)
        document = re.sub(' +', ' ', document)
        
        # simple porter stemming
        #document = ' '.join([ps.stem(word) for word in document.split()])
        
        # stopwords removal
        document = ' '.join([word for word in document.split() if word not in stop_words])
        
        return document

In [43]:
ad.fillna('n')

,genre
0,"['Documentary', 'Animation', 'Family']"
1,"['Crime', 'Horror', 'Thriller']"
2,['Family']
3,['Documentary']
4,['Thriller']
...,...
9821,"['Adventure', 'Family', 'Fantasy']"
9822,"['Documentary', 'Biography', 'History']"
9823,"['Sci-Fi', 'Thriller']"
9824,"['Drama', 'Music', 'Romance']"


In [44]:
l =[]
ar = list(ad[['genre']].iloc[:,0])
for i in ar:
    a = Preprocess.simple_text_preprocessor(i)
    l.append(a)
    
ad['genre']=[x for x in l]
ad.head()

,genre
0,documentary animation family
1,crime horror thriller
2,family
3,documentary
4,thriller


In [45]:
df[['title']].iloc[:,0]

0                     Dinosaur Planet
1                           Character
2                   Get Up and Dance!
3       The Rise and Fall of El Chapo
4                         Sick - IMDb
                    ...              
9821              All Monsters Attack
9822                     Fidel Castro
9823                            Epoch
9824                      The Company
9825                         Predator
Name: title, Length: 9826, dtype: object

In [46]:
ad['title'] = df[['title']].iloc[:,0]
ad.head()

,genre,title
0,documentary animation family,Dinosaur Planet
1,crime horror thriller,Character
2,family,Get Up and Dance!
3,documentary,The Rise and Fall of El Chapo
4,thriller,Sick - IMDb


In [47]:
'''from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(ad[['genre']].iloc[:,0])
cv_matrix = cv_matrix.toarray()
cv_matrix'''

"from sklearn.feature_extraction.text import CountVectorizer\n\ncv = CountVectorizer(min_df=0., max_df=1.)\ncv_matrix = cv.fit_transform(ad[['genre']].iloc[:,0])\ncv_matrix = cv_matrix.toarray()\ncv_matrix"

In [48]:
'''
# get all unique words in the corpus
vocab = cv.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)'''

'\n# get all unique words in the corpus\nvocab = cv.get_feature_names()\n# show document feature vectors\npd.DataFrame(cv_matrix, columns=vocab)'

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(ad[['genre']].iloc[:,0])
tv_matrix = tv_matrix.toarray()
#tv_matrix

In [50]:
vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,action,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,...,romance,sci,short,show,sport,talk,thriller,tv,war,western
0,0.00,0.00,0.61,0.00,0.0,0.00,0.51,0.0,0.61,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,0.00,0.00,0.00,0.00,0.0,0.58,0.00,0.0,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.53,0.0,0.0,0.0
2,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,1.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,0.00,0.00,0.00,0.00,0.0,0.00,1.00,0.0,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9821,0.00,0.55,0.00,0.00,0.0,0.00,0.00,0.0,0.59,0.59,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9822,0.00,0.00,0.00,0.63,0.0,0.00,0.45,0.0,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9823,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.60,0.0,0.0,0.0,0.0,0.52,0.0,0.0,0.0
9824,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.4,0.00,0.00,...,0.59,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [51]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,9816,9817,9818,9819,9820,9821,9822,9823,9824,9825
0,1.000000,0.000000,0.606505,0.513643,0.000000,0.000000,0.186282,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.357275,0.231742,0.000000,0.0,0.000000
1,0.000000,1.000000,0.000000,0.000000,0.525915,0.000000,0.000000,0.339921,0.000000,0.0,...,0.000000,0.496889,0.260111,0.000000,0.000000,0.000000,0.000000,0.275387,0.0,0.217546
2,0.606505,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.589071,0.000000,0.000000,0.0,0.000000
3,0.513643,0.000000,0.000000,1.000000,0.000000,0.000000,0.362668,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.451173,0.000000,0.0,0.000000
4,0.000000,0.525915,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.494586,0.000000,0.000000,0.000000,0.000000,0.523634,0.0,0.413653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9821,0.357275,0.000000,0.589071,0.000000,0.000000,0.000000,0.255384,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.251401
9822,0.231742,0.000000,0.000000,0.451173,0.000000,0.000000,0.163626,0.000000,0.397547,0.0,...,0.000000,0.000000,0.000000,0.000000,0.446447,0.000000,1.000000,0.000000,0.0,0.000000
9823,0.000000,0.275387,0.000000,0.000000,0.523634,0.728523,0.500201,0.640290,0.000000,0.0,...,0.000000,0.000000,0.944527,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.789965
9824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.094874,0.000000,0.119332,0.0,...,0.395903,0.204220,0.130028,0.395903,0.416943,0.000000,0.000000,0.000000,1.0,0.000000


In [52]:
from sklearn.cluster import KMeans
km_plus = KMeans(n_clusters=7,init='k-means++',n_init=10,max_iter=300,tol=1e-4,random_state=1)
y_km_plus = km_plus.fit_predict(similarity_matrix)
cluster_labels = km_plus.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
new_df= pd.concat([ad, cluster_labels,df[['rating']]], axis=1)
new_df.head()

,genre,title,ClusterLabel,rating
0,documentary animation family,Dinosaur Planet,6,7.7
1,crime horror thriller,Character,3,8.3
2,family,Get Up and Dance!,4,8.1
3,documentary,The Rise and Fall of El Chapo,6,6.9
4,thriller,Sick - IMDb,3,NaN


In [53]:
new_df.fillna(new_df.mean())

/var/folders/qr/06840hdx0_n24x3_vtrp47rm0000gn/T/ipykernel_6415/2768804178.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  new_df.fillna(new_df.mean())


,genre,title,ClusterLabel,rating
0,documentary animation family,Dinosaur Planet,6,7.700000
1,crime horror thriller,Character,3,8.300000
2,family,Get Up and Dance!,4,8.100000
3,documentary,The Rise and Fall of El Chapo,6,6.900000
4,thriller,Sick - IMDb,3,6.680635
...,...,...,...,...
9821,adventure family fantasy,All Monsters Attack,4,3.900000
9822,documentary biography history,Fidel Castro,6,6.400000
9823,sci fi thriller,Epoch,0,4.900000
9824,drama music romance,The Company,1,6.300000


In [54]:
'''distortions = []
for i in range(1, 15):
    km = KMeans(n_clusters=i, 
                init='k-means++', 
                n_init=10, 
                max_iter=300, 
                random_state=0)
    km.fit(similarity_matrix)
    distortions.append(km.inertia_)'''

"distortions = []\nfor i in range(1, 15):\n    km = KMeans(n_clusters=i, \n                init='k-means++', \n                n_init=10, \n                max_iter=300, \n                random_state=0)\n    km.fit(similarity_matrix)\n    distortions.append(km.inertia_)"

In [55]:
'''import matplotlib.pyplot as plt
plt.plot(range(1, 15), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.tight_layout()
#plt.savefig('images/11_03.png', dpi=300)
plt.show()'''

"import matplotlib.pyplot as plt\nplt.plot(range(1, 15), distortions, marker='o')\nplt.xlabel('Number of clusters')\nplt.ylabel('Distortion')\nplt.tight_layout()\n#plt.savefig('images/11_03.png', dpi=300)\nplt.show()"

In [61]:
text = str(input(""))
df_test = pd.DataFrame([text],columns =['genre'])
df_test.head()

,genre
0,action


In [62]:
c = list(df_test.iloc[:,0])
df_test['genre']=[Preprocess.simple_text_preprocessor(c[0])]
X_test_new = df_test[['genre']]
tv_test = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix_test = tv_test.fit_transform(df_test[['genre']].iloc[:,0])
tv_matrix_test = tv_matrix_test.toarray()
vocab = tv_test.get_feature_names()
df_test = pd.DataFrame(np.round(tv_matrix_test, 2), columns=vocab)
df_test.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,action
0,1.0


In [63]:
new_test_data=new_df.drop_duplicates(subset=['title'],keep='first')

In [64]:
from itertools import permutations 
st = [x for x in df_test.columns]
st1 = '|'.join(tuple(st))
l=[]
perm = permutations(st)
for i in list(perm):
    l.append(' '.join(tuple(i)))
l
s = '|'.join(l)

#s_new = s+'|'+st1

fil = new_test_data[new_test_data['genre'].str.contains(s)]

#fil2 = new_test_data[new_test_data['genre'].str.contains(s_new)]
#fil3 = new_test_data[new_test_data['genre'].str.contains(s_new)]
#a = pd.concat([fil,fil2.to_frame().T],ignore_index=True)


((fil.loc[(fil.rating>=6)].sort_values(by = ['rating','ClusterLabel'], ascending=False).groupby(['ClusterLabel']))).head()

,genre,title,ClusterLabel,rating
7655,action drama history war,Band of Brothers,4,9.4
4812,action adventure drama fantasy,Game of Thrones,0,9.2
27,short action drama,Silent Service,4,9.1
7869,animation action adventure comedy drama fantas...,Fullmetal Alchemist: Brotherhood,0,9.1
8145,action drama fantasy history romance,Word of Honor,1,9.0
221,animation action adventure family sci fi,Batman: The Animated Series,0,9.0
1070,action adventure drama fantasy,The Lord of the Rings: The Return of the King,0,8.9
2253,animation action adventure fantasy thriller,Demon Slayer: Kimetsu no Yaiba,0,8.7
7684,animation action adventure drama history,Rurouni Kenshin: Trust and Betrayal,4,8.6
805,action drama mystery,Hara-Kiri,3,8.6


In [65]:
s_new = s+'|'+st1
s_new

'action|action'